## import libraries

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import GRU, Input, Dense, Activation, RepeatVector,BatchNormalization, Bidirectional, LSTM, Dropout, Embedding
from sklearn.model_selection import train_test_split
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.callbacks import EarlyStopping
from keras.models import Model
from keras.layers import Input, Reshape, Conv2D, MaxPooling2D, Flatten, Dense, LSTM, Embedding, Dropout, BatchNormalization
from sklearn.metrics import confusion_matrix
from nltk.tokenize import word_tokenize
import matplotlib.pyplot as plt
from collections import Counter
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.utils import shuffle
from keras.preprocessing.image import ImageDataGenerator


import pandas as pd
import re
import string
import csv

import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from keras.callbacks import ReduceLROnPlateau
import os
from keras import backend as K
import numpy as np
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, GlobalAveragePooling2D, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
from google.colab import files
import pandas as pd

import pickle
import shutil
from nltk.corpus import stopwords
from textblob import TextBlob
from textblob import Word
import nltk
from PIL import Image

#Preprocessing layers

In [ ]:
dir ="/content/drive/MyDrive/Emotion"

In [ ]:
import os
import cv2
import numpy as np

# Define the path to the directory containing the image folders
image_dir = dir

# Initialize empty lists to store images and corresponding labels
images = []
labels = []

# Get a sorted list of subdirectories in alphabetical order
subdirectories = sorted(os.listdir(image_dir))

# Create a mapping from folder names to numerical labels
label_mapping = {}

# Define the target shape for all images (e.g., 224x224 pixels for consistency)
target_shape = (224, 224)

# Iterate through the subdirectories in alphabetical order
for index, label in enumerate(subdirectories):
    label_dir = os.path.join(image_dir, label)

    # Check if it's a directory
    if os.path.isdir(label_dir):
        # Iterate through the files in the subdirectory
        for image_file in os.listdir(label_dir):
            image_path = os.path.join(label_dir, image_file)

            # Load the image using OpenCV
            image = cv2.imread(image_path)

            # Check if the image loaded successfully
            if image is not None:
                # Resize the image to the target shape (e.g., 224x224)
                image = cv2.resize(image, target_shape)

                # Normalize the image by dividing by 255 to scale pixel values between 0 and 1
                image = image / 255.0

                # Append the image and corresponding label to the lists
                images.append(image)
                labels.append(index)  # Use the index as the numerical label
                label_mapping[index] = label

# Convert lists to NumPy arrays for further processing (if needed)
images = np.array(images)
labels = np.array(labels)

# Now, you have your images with a common shape (e.g., 224x224) and normalized pixel values between 0 and 1.
# You can use label_mapping to get the original folder names.


In [ ]:
len(labels)

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Emotion_csv/Emotion.csv")
df['Clean_Text'] = df['Clean_Text'].astype(str)
data= df.iloc[1:8122]
data.head(4 )

In [ ]:
# Separate data by label
label_0_data = data[data['label'] == 0].sample(n=390, random_state=42)
label_1_data = data[data['label'] == 1].sample(n=390, random_state=42)
label_2_data = data[data['label'] == 2].sample(n=390, random_state=42)

# Concatenate the balanced data for all labels
balanced_data = pd.concat([label_0_data, label_1_data, label_2_data])

# Shuffle the balanced data
balanced_data = balanced_data.sample(frac=1, random_state=42)

In [ ]:
balanced_data.head(3)

In [ ]:

def preprocess_images_from_folder(main_folder, target_size=(48, 48), images_per_class=390):
    image_data = []
    labels = []

    class_labels = sorted(os.listdir(main_folder))  # Get the class labels from folder names

    for class_label in class_labels:
        class_folder = os.path.join(main_folder, class_label)
        images_loaded = 0  # Counter to track the number of loaded images per class
        for filename in os.listdir(class_folder):
            if images_loaded >= images_per_class:
                break  # Stop loading images for this class if the limit is reached

            img_path = os.path.join(class_folder, filename)
            img = Image.open(img_path)
            img = img.resize(target_size)
            img_array = np.array(img)
            img_array = img_array.astype('float32') / 255.0
            image_data.append(img_array)

            # Convert class label to an integer index
            label_index = class_labels.index(class_label)
            labels.append(label_index)

            images_loaded += 1

    image_data = np.array(image_data)
    labels = np.array(labels)

    # Convert integer labels to one-hot encoded vectors
    labels_one_hot = to_categorical(labels)

    return image_data, labels_one_hot

# Replace 'main_folder' with the path to your main folder containing class folders
main_folder = dir
target_image_size = (48, 48)
images_per_class = 390

# Preprocess limited number of images from the folder
image_data, labels_one_hot = preprocess_images_from_folder(main_folder, target_image_size, images_per_class)

In [ ]:
labels_one_hot.shape

In [ ]:
happy_class_index = 0
happy_count = np.sum(labels_one_hot[:, happy_class_index])
print("Number of 'depression' images:", happy_count)

In [ ]:
def preprocess_text_data(text_data, labels, max_sequence_length):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(text_data)

    # Convert text data to sequences of integer indices
    sequences = tokenizer.texts_to_sequences(text_data)

    # Pad sequences to a fixed length
    padded_sequences = pad_sequences(sequences,
    maxlen=max_sequence_length,
    padding='pre',
    truncating='pre')

    max_index = np.max(padded_sequences)
    vocabulary_size = max_index + 1


    # Convert class labels to one-hot encoded vectors
    labels_one_hot = to_categorical(labels)

    return padded_sequences, labels_one_hot, vocabulary_size


text_data = balanced_data["Clean_Text"]
labels =balanced_data["label"]
max_sequence_length = 31
# Preprocess the text data and combine it with the image data
padded_sequences, label2_one_hot, vocabulary_size = preprocess_text_data(text_data, labels, max_sequence_length)

In [ ]:
max_index = np.max(padded_sequences)
vocabulary_size = max_index + 1
label2_one_hot.astype

In [ ]:

num_image_samples = image_data.shape[0]
num_text_samples = padded_sequences.shape[0]


X_image_train, X_image_val, X_text_train, X_text_val, y_label1_train, y_label1_val, y_label2_train, y_label2_val = train_test_split(
    image_data, padded_sequences, labels_one_hot , label2_one_hot, test_size=0.2, random_state=42
)

# Combine of model

In [ ]:

# CNN model for grayscale images
cnn_input_shape = (48,48,3)
lstm_input_shape = 31
cnn_input = Input(shape=cnn_input_shape)

cnn_layer1 = Conv2D(32, (3, 3), activation='relu', padding='same')(cnn_input)
cnn_layer2 = MaxPooling2D((2, 2))(cnn_layer1)
cnn_layer3 = Conv2D(64, (3, 3), activation='relu', padding='same')(cnn_layer2)
cnn_layer4 = MaxPooling2D((2, 2))(cnn_layer3)
cnn_layer5 = Conv2D(128, (3, 3), activation='relu', padding='same')(cnn_layer4)
cnn_layer6 = MaxPooling2D((2, 2))(cnn_layer5)

cnn_gap = GlobalAveragePooling2D()(cnn_layer6)

cnn_dense1 = Dense(256, activation='relu')(cnn_gap)
cnn_dropout1 = Dropout(0.5)(cnn_dense1)
cnn_output = Dense(3, activation='softmax')(cnn_dropout1)

# LSTM model
lstm_input = Input(shape=lstm_input_shape)
embedding = Embedding(input_dim=vocabulary_size, output_dim=100)(lstm_input)
lstm_layer1 = LSTM(64, return_sequences=True, dropout=0.3, recurrent_dropout=0.3)(embedding)
lstm_layer2 = LSTM(32, dropout=0.3, recurrent_dropout=0.3)(lstm_layer1)
lstm_batchnorm = BatchNormalization()(lstm_layer2)

lstm_output = Dense(3, activation='softmax')(lstm_batchnorm)
# Combined architecture
model = Model(inputs=[cnn_input, lstm_input], outputs=[cnn_output, lstm_output])
model.summary()

# Compile and train the model
model.compile(optimizer='adam', loss=['categorical_crossentropy', 'categorical_crossentropy'], metrics=['accuracy'])

In [ ]:
cnn_input_shape = (48, 48, 1)
lstm_input_shape = 31
cnn_input = Input(shape=cnn_input_shape)

cnn_layer1 = Conv2D(32, (3, 3), activation='relu', padding='same')(cnn_input)
cnn_layer1_bn = BatchNormalization()(cnn_layer1)
cnn_layer2 = MaxPooling2D((2, 2))(cnn_layer1_bn)
cnn_layer3 = Conv2D(64, (3, 3), activation='relu', padding='same')(cnn_layer2)
cnn_layer3_bn = BatchNormalization()(cnn_layer3)
cnn_layer4 = MaxPooling2D((2, 2))(cnn_layer3_bn)
cnn_layer5 = Conv2D(128, (3, 3), activation='relu', padding='same')(cnn_layer4)
cnn_layer5_bn = BatchNormalization()(cnn_layer5)
cnn_layer6 = MaxPooling2D((2, 2))(cnn_layer5_bn)

cnn_gap = GlobalAveragePooling2D()(cnn_layer6)

cnn_dense1 = Dense(256, activation='relu')(cnn_gap)
cnn_dropout1 = Dropout(0.5)(cnn_dense1)
cnn_output = Dense(3, activation='softmax')(cnn_dropout1)

# LSTM model
lstm_input = Input(shape=lstm_input_shape)
embedding = Embedding(input_dim=vocabulary_size, output_dim=100)(lstm_input)
lstm_layer1 = LSTM(64, return_sequences=True, dropout=0.3, recurrent_dropout=0.3)(embedding)
lstm_layer1_bn = BatchNormalization()(lstm_layer1)
lstm_layer2 = LSTM(32, dropout=0.3, recurrent_dropout=0.3)(lstm_layer1_bn)

lstm_output = Dense(3, activation='softmax')(lstm_layer2)

# Combined architecture
model = Model(inputs=[cnn_input, lstm_input], outputs=[cnn_output, lstm_output])
model.summary()

# Compile and train the model
model.compile(optimizer='adam', loss=['categorical_crossentropy', 'categorical_crossentropy'], metrics=['accuracy'])

In [ ]:
# Fit the multi-modal model
history = model.fit(
    [X_image_train, X_text_train],  # Input data for both branches (images and text)
    [y_label1_train, y_label2_train],  # Labels for both output layers (label 1 and label 2)
    validation_data=([X_image_val, X_text_val], [y_label1_val, y_label2_val]),  # Validation data
    batch_size=15,
    epochs=30,  # Set the number of epochs for training
    verbose=1  # Set verbosity level (0: silent, 1: progress bar, 2: one line per epoch)
)

In [ ]:
model_path = '/content/drive/MyDrive/combine_model_30.h5'

# Save the model
#model.save(model_path)

print("Model saved successfully.")

In [ ]:
model =load_model(model_path)

# Evaluation of the model

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, f1_score
import numpy as np

In [ ]:
X_image_val = X_image_val  # Validation image data
X_text_val = X_text_val   # Validation text data

# Assuming you have separate true labels for both image and text
y_image_val =  y_label1_val  # Validation labels for image category
y_text_val =   y_label2_val # Validation labels for text category

# #Make predictions using the model
y_pred_probabilities = model.predict([X_image_val, X_text_val])

# Convert probabilities to binary labels for both image and text
y_pred_image = (y_pred_probabilities[0] > 0.5).astype(int)
y_pred_text = (y_pred_probabilities[1] > 0.5).astype(int)

# Calculate accuracy for both image and text labels
accuracy_image = accuracy_score(y_image_val, y_pred_image)
accuracy_text = accuracy_score(y_text_val, y_pred_text)

print("Validation Accuracy for Image Labels:", int(accuracy_image*100),'%')
print("Validation Accuracy for Text Labels:", int(accuracy_text*100),'%')

In [ ]:
from sklearn.metrics import hamming_loss, jaccard_score, f1_score

# Calculate Hamming loss for image and text labels
hamming_loss_image = hamming_loss(y_image_val, y_pred_image)
hamming_loss_text = hamming_loss(y_text_val, y_pred_text)

# Calculate Jaccard similarity score for image and text labels
jaccard_image = jaccard_score(y_image_val, y_pred_image, average='samples')
jaccard_text = jaccard_score(y_text_val, y_pred_text, average='samples')

# Calculate F1-score for image and text labels
f1_score_image = f1_score(y_image_val, y_pred_image, average='samples')
f1_score_text = f1_score(y_text_val, y_pred_text, average='samples')

print("Hamming Loss for Image Labels:", hamming_loss_image)
print("Hamming Loss for Text Labels:", hamming_loss_text)

print("Jaccard Similarity Score for Image Labels:", jaccard_image)
print("Jaccard Similarity Score for Text Labels:", jaccard_text)

print("F1-score for Image Labels:", f1_score_image)
print("F1-score for Text Labels:", f1_score_text)

# Calculate the number of correct predictions for image labels
correct_image_predictions = sum((y_pred_image == y_image_val).all(axis=1))

# Calculate the number of correct predictions for text labels
correct_text_predictions = sum((y_pred_text == y_text_val).all(axis=1))

print("Number of Correct Predictions for Image Labels:", correct_image_predictions)
print("Number of Correct Predictions for Text Labels:", correct_text_predictions)

# Calculate the number of incorrect predictions for image labels
incorrect_image_predictions = len(y_image_val) - correct_image_predictions

# Calculate the number of incorrect predictions for text labels
incorrect_text_predictions = len(y_text_val) - correct_text_predictions

print("Number of Incorrect Predictions for Image Labels:", incorrect_image_predictions)
print("Number of Incorrect Predictions for Text Labels:", incorrect_text_predictions)

In [ ]:
# Plotting
labels = ['Image Labels', 'Text Labels']
correct_predictions = [correct_image_predictions, correct_text_predictions]
incorrect_predictions = [incorrect_image_predictions, incorrect_text_predictions]

x = range(len(labels))

plt.bar(x, correct_predictions, label='Correct Predictions', color='green')
plt.bar(x, incorrect_predictions, bottom=correct_predictions, label='Incorrect Predictions', color='red')

plt.xlabel('Label Type')
plt.ylabel('Number of Predictions')
plt.title('Number of Correct and Incorrect Predictions')
plt.xticks(x, labels)
plt.legend()
plt.show()

In [ ]:
train_image_losses = []
train_image_accuracies = []
train_text_losses = []
train_text_accuracies = []

# training data
image_train_data = X_image_train
text_train_data =    X_text_train
label_image_train_data = y_label1_train
label_text_train_data = y_label2_train


# Validation data for both modalities
image_val_data = X_image_val
text_val_data = X_text_val
label_image_val_data = y_label1_val
label_text_val_data = y_label2_val

# Training loop
epochs = 25
for epoch in range(epochs):
    # Train the multi-modal model
    history = model.fit([image_train_data, text_train_data], [label_image_train_data, label_text_train_data],
                                    validation_data=([image_val_data, text_val_data],
                                                     [label_image_val_data, label_text_val_data]),
                                    epochs=1, batch_size=32)

    # Append metrics to lists for image modality
    train_image_losses.append(history.history['dense_7_loss'][0])
    train_image_accuracies.append(history.history['dense_7_accuracy'][0])

    # Append metrics to lists for text modality
    train_text_losses.append(history.history['dense_8_loss'][0])
    train_text_accuracies.append(history.history['dense_8_accuracy'][0])

    # Plot performance curves using scatter plots
    plt.figure(figsize=(10, 4))

    plt.subplot(1, 2, 1)
    plt.scatter(range(epoch + 1), train_image_losses, label='Train Image Loss', marker='o')
    plt.scatter(range(epoch + 1), train_text_losses, label='Train Text Loss', marker='o')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.scatter(range(epoch + 1), train_image_accuracies, label='Train Image Accuracy', marker='o')
    plt.scatter(range(epoch + 1), train_text_accuracies, label='Train Text Accuracy', marker='o')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.tight_layout()
    plt.savefig(f'performance_plots_epoch_{epoch + 1}.png')  # Save plot as an image
    plt.close()

# Save the performance data to a CSV file
data = list(zip(range(epochs), train_image_losses, train_image_accuracies, train_text_losses, train_text_accuracies))
csv_file = 'performance_data.csv'
with open(csv_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Epoch', 'Train Image Loss', 'Train Image Accuracy', 'Train Text Loss', 'Train Text Accuracy'])
    writer.writerows(data)

print(f'Data saved to {csv_file}')

In [ ]:
image = precision_score(y_image_val, y_pred_image, average = 'micro')
Text = precision_score(y_text_val, y_pred_text, average = 'micro')

# Calculate accuracy
#accuracy = accuracy_score(true_labels, predicted_classes)

# Calculate precision
#precision = , predicted_classes, average='weighted')

# Calculate F1 score
#f1 = f1_score(true_labels, predicted_classes, average='weighted')

#print("Confusion Matrix:")
print("image_precision_score",image)
print("text_precision_score",Text)
#print("Accuracy:", accuracy)
#print("Precision:", precision)
#print("F1 Score:", f1)